<a href="https://colab.research.google.com/github/drkoba/high-freq-word-for-res-trends/blob/main/high-frequency%20analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import io
import re
import os
from google.colab import files


In [ ]:
# Step 1: Upload files
print("Please upload the Excel or CSV file you want to analyze.")
uploaded = files.upload()

Please upload the Excel or CSV file you want to analyze.


Saving 頻出語_NPS_全年代_解析用 2（旧ストップワード）.xlsx to 頻出語_NPS_全年代_解析用 2（旧ストップワード）.xlsx


In [ ]:
# Step 2: Set user input
# Specify the periods to analyze as a list (e.g., ['1990-1999', '2000-2009', '2010-2019', '2020-2024'])
# You can add or remove periods as desired
periods = ['1990-1999', '2000-2009', '2010-2019', '2020-2024']

In [ ]:
# Specify the top N words to select (e.g., 50)
top_n = 5000

In [ ]:
# Step 3: Choose output format
print("\nPlease select the format to save:")
print("1: CSV")
print("2: Excel")
choice = input("Please enter 1 or 2: ")

if choice == '1':
    output_format = 'csv'
elif choice == '2':
    output_format = 'excel'
else:
    print("Invalid selection. CSV format will be selected by default.")
    output_format = 'csv'



Please select the format to save:
1: CSV
2: Excel
Please enter 1 or 2: 1


In [ ]:
# Step 4: Read file and check column names
# Get the name of the uploaded file
file_name = list(uploaded.keys())[0]
print(f'\nUploaded file name: {file_name}')


Uploaded file name: 頻出語_NPS_全年代_解析用 2（旧ストップワード）.xlsx


In [ ]:
# Get the file extension
file_extension = file_name.split('.')[-1].lower()

In [ ]:
# Load data (in case of Excel or CSV file)
try:
    if file_extension == 'csv':
        df = pd.read_csv(io.BytesIO(uploaded[file_name]))
    elif file_extension in ['xls', 'xlsx']:
        df = pd.read_excel(io.BytesIO(uploaded[file_name]))
    else:
        raise ValueError("Unsupported file format. Please upload a CSV or Excel file.")
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    raise

# Remove leading and trailing spaces from column names
df.columns = df.columns.str.strip()

# Convert full-width brackets to half-width brackets (if necessary)
df.columns = df.columns.str.replace('（', '(', regex=False).str.replace('）', ')', regex=False)

# Check data (display first few rows)
print("\nDataFrame column names:")
print(df.columns.tolist())

print("\nTop part of the data:")
print(df.head())



DataFrame column names:
['word(1990-1999)', 'count(1990-1999)', 'word(2000-2009)', 'count(2000-2009)', 'word(2010-2019)', 'count(2010-2019)', 'word(2020-2024)', 'count(2020-2024)']

Top part of the data:
  word(1990-1999)  count(1990-1999) word(2000-2009)  count(2000-2009)  \
0            drug             178.0            drug             663.0   
1        receptor             124.0            MDMA             315.0   
2     cannabinoid              94.0        receptor             277.0   
3         binding              90.0          method             249.0   
4             rat              53.0     cannabinoid             215.0   

  word(2010-2019)  count(2010-2019) word(2020-2024)  count(2020-2024)  
0            drug            7036.0            drug              4990  
1       synthetic            3742.0       synthetic              2748  
2     cannabinoid            2571.0             NPS              2478  
3          method            2540.0          method              216

In [ ]:
# Step 5: Mapping column names
# Function to search for similar column names
def find_similar_columns(df_columns, keyword, period):
    # Allow full-width and half-width brackets, and allow spaces
    pattern = re.compile(rf'{keyword}\s*[\(\（]\s*{period}\s*[\)\）]', re.IGNORECASE)
    matches = [col for col in df_columns if pattern.match(col)]
    if not matches:
        # Output column names for debugging
        print(f"No column matching '{keyword} ({period})' was found.")
        print("List of available column names:")
        for col in df_columns:
            print(f" - {col}")
    return matches[0] if matches else None

In [ ]:
# Map actual column names for each period
words_columns = []
counts_columns = []

for period in periods:
    word_col = find_similar_columns(df.columns, 'word', period)
    count_col = find_similar_columns(df.columns, 'count', period)
    if word_col and count_col:
        words_columns.append(word_col)
        counts_columns.append(count_col)
        print(f"{period}: '{word_col}', '{count_col}'")
    else:
        print(f"{period}: Corresponding columns not found, skipping data.")
        words_columns.append(None)
        counts_columns.append(None)

1990-1999: 'word(1990-1999)', 'count(1990-1999)'
2000-2009: 'word(2000-2009)', 'count(2000-2009)'
2010-2019: 'word(2010-2019)', 'count(2010-2019)'
2020-2024: 'word(2020-2024)', 'count(2020-2024)'


In [ ]:
# Step 6: Data shaping and analysis
# Split data by each period and merge
data_frames = []

for period, word_col, count_col in zip(periods, words_columns, counts_columns):
    if word_col and count_col:
        try:
            temp_df = df[[word_col, count_col]].copy()
            temp_df.rename(columns={word_col: 'word', count_col: 'count'}, inplace=True)
            temp_df['period'] = period
            data_frames.append(temp_df)
            print(f"Successfully added data for {period}.")
        except KeyError as e:
            print(f"Column {e} not found. Skipping data for {period}.")
    else:
        print(f"Corresponding columns do not exist for {period}, skipping data.")


Successfully added data for 1990-1999.
Successfully added data for 2000-2009.
Successfully added data for 2010-2019.
Successfully added data for 2020-2024.


In [ ]:
# Step 7: Save and download results
if data_frames:
    from google.colab import files

    # Save and download new terms
    print("\nStarting to save and download new terms.")
    for period in new_terms_top_n:
        if new_terms_top_n[period]:  # If the list is not empty
            df_new = pd.DataFrame({
                'word': new_terms_top_n[period],
                'count': [keywords[period].get(term, 0) for term in new_terms_top_n[period]]
            })
            if output_format.lower() == 'csv':
                output_filename = f'new_terms_{period}.csv'
                df_new.to_csv(output_filename, index=False, encoding='utf-8-sig')  # 'utf-8-sig' for better Excel compatibility
            elif output_format.lower() == 'excel':
                output_filename = f'new_terms_{period}.xlsx'
                df_new.to_excel(output_filename, index=False)
            else:
                print(f"Unsupported format: {output_format}. Skipping {period}.")
                continue
            files.download(output_filename)
            print(f"New Terms ({period}) saved and downloaded as '{output_filename}'.")
        else:
            print(f"No new terms found for {period}.")

    # Save and download growth terms
    print("\nStarting to save and download Growth Terms.")
    for period in growth_terms:
        if growth_terms[period]:  # If the list is not empty
            df_growth = pd.DataFrame({
                'word': growth_terms[period],
                'growth_rate (%)': [growth_rates[period].get(term, 0) for term in growth_terms[period]],
                'count': [growth_counts[period].get(term, 0) for term in growth_terms[period]]
            })
            if output_format.lower() == 'csv':
                output_filename = f'growth_terms_{period}.csv'
                df_growth.to_csv(output_filename, index=False, encoding='utf-8-sig')  # 'utf-8-sig' for better Excel compatibility
            elif output_format.lower() == 'excel':
                output_filename = f'growth_terms_{period}.xlsx'
                df_growth.to_excel(output_filename, index=False)
            else:
                print(f"Unsupported format: {output_format}. Skipping {period}.")
                continue
            files.download(output_filename)
            print(f"Growth Terms ({period}) saved and downloaded as '{output_filename}'.")
        else:
            print(f"No growth terms found for {period}.")

    print("\nExtraction and saving of New Terms and Growth Terms are complete.")
else:
    print("The DataFrame is empty. Skipping saving results.")


Starting to save and download new terms.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

New Terms (2000-2009) saved and downloaded as 'new_terms_2000-2009.csv'.


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Growth Terms (2010-2019) saved and downloaded as 'growth_terms_2010-2019.csv'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Growth Terms (2020-2024) saved and downloaded as 'growth_terms_2020-2024.csv'.

Extraction and saving of New Terms and Growth Terms are complete.
